In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re

In [2]:
complaints_raw = pd.read_csv("NYPD_Complaint_Year_To_Date.csv")
complaints_raw.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,343720611,48.0,BRONX,06/25/2019,14:15:00,06/25/2019,14:20:00,COMPLETED,NaN,NaN,...,M,NaN,25-44,BLACK,F,1016986.0,250197.0,40.853355,-73.881668,"(40.85335545600003, -73.88166837799997)"
1,953123251,14.0,MANHATTAN,06/25/2019,13:55:00,NaN,NaN,COMPLETED,NaN,NaN,...,M,NaN,UNKNOWN,UNKNOWN,D,987220.0,212676.0,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)"
2,553754783,18.0,MANHATTAN,06/25/2019,13:20:00,06/25/2019,13:25:00,COMPLETED,NaN,NaN,...,NaN,NaN,45-64,WHITE,M,991199.0,216774.0,40.761677,-73.974917,"(40.761676504000036, -73.97491700299997)"
3,398494230,52.0,BRONX,06/25/2019,13:20:00,NaN,NaN,ATTEMPTED,NaN,NaN,...,M,NaN,25-44,WHITE HISPANIC,F,1017542.0,255919.0,40.869059,-73.879630,"(40.86905853200005, -73.87963014799993)"
4,219893130,19.0,MANHATTAN,06/25/2019,13:20:00,06/25/2019,13:36:00,COMPLETED,NaN,NaN,...,M,NaN,45-64,BLACK,M,998329.0,224701.0,40.783426,-73.949163,"(40.78342557000008, -73.94916258199999)"


In [3]:
# Selction of only the interesting columns for the study
complaints_raw_selection = complaints_raw[["ADDR_PCT_CD", "BORO_NM", "CMPLNT_FR_DT", "CMPLNT_FR_TM", "KY_CD", "LAW_CAT_CD", "OFNS_DESC", "SUSP_AGE_GROUP", "SUSP_RACE", "SUSP_SEX", "VIC_AGE_GROUP", "VIC_RACE", "VIC_SEX", "Latitude", "Longitude", "Lat_Lon"]]
complaints_raw_selection.head()

,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,KY_CD,LAW_CAT_CD,OFNS_DESC,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Latitude,Longitude,Lat_Lon
0,48.0,BRONX,06/25/2019,14:15:00,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,M,25-44,BLACK,F,40.853355,-73.881668,"(40.85335545600003, -73.88166837799997)"
1,14.0,MANHATTAN,06/25/2019,13:55:00,341,MISDEMEANOR,PETIT LARCENY,45-64,BLACK,M,UNKNOWN,UNKNOWN,D,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)"
2,18.0,MANHATTAN,06/25/2019,13:20:00,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,NaN,NaN,NaN,45-64,WHITE,M,40.761677,-73.974917,"(40.761676504000036, -73.97491700299997)"
3,52.0,BRONX,06/25/2019,13:20:00,104,FELONY,RAPE,45-64,BLACK HISPANIC,M,25-44,WHITE HISPANIC,F,40.869059,-73.879630,"(40.86905853200005, -73.87963014799993)"
4,19.0,MANHATTAN,06/25/2019,13:20:00,578,VIOLATION,HARRASSMENT 2,25-44,BLACK,M,45-64,BLACK,M,40.783426,-73.949163,"(40.78342557000008, -73.94916258199999)"


In [4]:
# Changing the columns names
complaints_raw_selection = complaints_raw_selection.rename(columns={"ADDR_PCT_CD":"Address_precinct", "BORO_NM":"Hood", "CMPLNT_FR_DT":"Complaint_date", "CMPLNT_FR_TM":"Complaint_time", "KY_CD":"Key_code", "LAW_CAT_CD":"Offense_level", "OFNS_DESC":"Offense_desc", "SUSP_AGE_GROUP":"Suspect_age", "SUSP_RACE":"Suspect_race", "SUSP_SEX":"Suspect_sex", "VIC_AGE_GROUP":"Victim_age", "VIC_RACE":"Victim_race", "VIC_SEX":"Victim_sex"})
complaints_raw_selection.head()

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon
0,48.0,BRONX,06/25/2019,14:15:00,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,M,25-44,BLACK,F,40.853355,-73.881668,"(40.85335545600003, -73.88166837799997)"
1,14.0,MANHATTAN,06/25/2019,13:55:00,341,MISDEMEANOR,PETIT LARCENY,45-64,BLACK,M,UNKNOWN,UNKNOWN,D,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)"
2,18.0,MANHATTAN,06/25/2019,13:20:00,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,NaN,NaN,NaN,45-64,WHITE,M,40.761677,-73.974917,"(40.761676504000036, -73.97491700299997)"
3,52.0,BRONX,06/25/2019,13:20:00,104,FELONY,RAPE,45-64,BLACK HISPANIC,M,25-44,WHITE HISPANIC,F,40.869059,-73.879630,"(40.86905853200005, -73.87963014799993)"
4,19.0,MANHATTAN,06/25/2019,13:20:00,578,VIOLATION,HARRASSMENT 2,25-44,BLACK,M,45-64,BLACK,M,40.783426,-73.949163,"(40.78342557000008, -73.94916258199999)"


In [5]:
# Checking the types
complaints_raw_selection.dtypes

Address_precinct    float64
Hood                 object
Complaint_date       object
Complaint_time       object
Key_code              int64
Offense_level        object
Offense_desc         object
Suspect_age          object
Suspect_race         object
Suspect_sex          object
Victim_age           object
Victim_race          object
Victim_sex           object
Latitude            float64
Longitude           float64
Lat_Lon              object
dtype: object

In [6]:
complaints = complaints_raw_selection

In [7]:
# Change the Complaint_date and the Complaint_time to a datetime type
complaints = complaints.astype({"Complaint_date": "datetime64"})

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1029-05-22 00:00:00

In [8]:
# find where we have the 1029 in the Complaint_date
test = complaints[complaints['Complaint_date'].str.contains("1029")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon
45491,81.0,BROOKLYN,05/22/1029,09:00:00,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,25-44,BLACK,M,25-44,BLACK,F,40.680347,-73.930035,"(40.68034722300007, -73.93003524099998)"
163932,108.0,QUEENS,02/13/1029,23:00:00,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,UNKNOWN,UNKNOWN,U,45-64,WHITE HISPANIC,F,40.735927,-73.935949,"(40.735926660000075, -73.93594878299997)"


In [9]:
# We assume it's a typo that meant 2019 so we replace these 1029 by 2019 using regex
complaints["Complaint_date"] = complaints["Complaint_date"].apply(lambda x: re.sub("1029", "2019", x))

In [10]:
# Check again if the problem is solved
test = complaints[complaints['Complaint_date'].str.contains("1029")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon


In [11]:
# Try to change the Complaint_date type to datetime again
complaints = complaints.astype({"Complaint_date": "datetime64"})

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1019-04-22 00:00:00

In [12]:
# find where we have the 1019 in the Complaint_date
test = complaints[complaints['Complaint_date'].str.contains("1019")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon
84050,104.0,QUEENS,04/22/1019,01:10:00,107,FELONY,BURGLARY,UNKNOWN,UNKNOWN,U,UNKNOWN,UNKNOWN,D,40.727008,-73.893572,"(40.727007769000075, -73.89357155599998)"
90074,42.0,BRONX,04/14/1019,22:00:00,109,FELONY,GRAND LARCENY,NaN,NaN,NaN,25-44,BLACK HISPANIC,F,40.828691,-73.908354,"(40.82869107100004, -73.90835406399998)"
93446,84.0,BROOKLYN,04/14/1019,07:00:00,578,VIOLATION,HARRASSMENT 2,UNKNOWN,ASIAN / PACIFIC ISLANDER,M,45-64,BLACK,M,40.693657,-73.983323,"(40.693657467000044, -73.98332318799999)"
135374,73.0,BROOKLYN,03/11/1019,07:10:00,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,UNKNOWN,M,45-64,ASIAN / PACIFIC ISLANDER,M,40.673988,-73.911151,"(40.67398826900006, -73.91115146199996)"
173024,5.0,MANHATTAN,02/16/1019,13:00:00,578,VIOLATION,HARRASSMENT 2,45-64,WHITE HISPANIC,M,25-44,BLACK,F,40.716465,-74.002321,"(40.71646490500007, -74.00232096899998)"
186941,77.0,BROOKLYN,01/13/1019,06:40:00,113,FELONY,FORGERY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,40.670956,-73.954072,"(40.670955726000045, -73.95407163699997)"
199252,18.0,MANHATTAN,01/14/1019,12:20:00,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,BLACK,F,UNKNOWN,UNKNOWN,D,40.767955,-73.982362,"(40.76795511000005, -73.982362342)"


In [13]:
# We assume it's a typo that meant 2019 so we replace these 1019 by 2019 using regex
complaints["Complaint_date"] = complaints["Complaint_date"].apply(lambda x: re.sub("1019", "2019", x))

In [14]:
# Check again if the problem is solved
test = complaints[complaints['Complaint_date'].str.contains("1019")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon


In [15]:
# Try to change the Complaint_date type to datetime again
complaints = complaints.astype({"Complaint_date": "datetime64"})

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1018-12-10 00:00:00

In [16]:
# find where we have the 1018 in the Complaint_date
test = complaints[complaints['Complaint_date'].str.contains("1018")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon
218542,114.0,QUEENS,12/10/1018,17:10:00,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,UNKNOWN,U,25-44,ASIAN / PACIFIC ISLANDER,F,40.755280,-73.936291,"(40.75528008700008, -73.93629111799999)"
221204,70.0,BROOKLYN,12/14/1018,16:00:00,341,MISDEMEANOR,PETIT LARCENY,NaN,NaN,NaN,45-64,WHITE,M,40.615888,-73.957214,"(40.61588830300008, -73.95721436999997)"


In [17]:
# We assume it's a typo that meant 2019 so we replace these 1019 by 2019 using regex
complaints["Complaint_date"] = complaints["Complaint_date"].apply(lambda x: re.sub("1018", "2018", x))

In [18]:
# Check again if the problem is solved
test = complaints[complaints['Complaint_date'].str.contains("1018")]
test

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon


In [19]:
# Try to change the Complaint_date type to datetime again
complaints = complaints.astype({"Complaint_date": "datetime64"})

In [20]:
# Check the type for Complaint_date has been changed
complaints.dtypes

Address_precinct           float64
Hood                        object
Complaint_date      datetime64[ns]
Complaint_time              object
Key_code                     int64
Offense_level               object
Offense_desc                object
Suspect_age                 object
Suspect_race                object
Suspect_sex                 object
Victim_age                  object
Victim_race                 object
Victim_sex                  object
Latitude                   float64
Longitude                  float64
Lat_Lon                     object
dtype: object

In [21]:
# Change the type of Complaint_time to datetime
complaints = complaints.astype({"Complaint_time": "datetime64"})

In [22]:
# Check the type for Complaint_time has been changed
complaints.dtypes

Address_precinct           float64
Hood                        object
Complaint_date      datetime64[ns]
Complaint_time      datetime64[ns]
Key_code                     int64
Offense_level               object
Offense_desc                object
Suspect_age                 object
Suspect_race                object
Suspect_sex                 object
Victim_age                  object
Victim_race                 object
Victim_sex                  object
Latitude                   float64
Longitude                  float64
Lat_Lon                     object
dtype: object

In [26]:
complaints.head()

,Address_precinct,Hood,Complaint_date,Complaint_time,Key_code,Offense_level,Offense_desc,Suspect_age,Suspect_race,Suspect_sex,Victim_age,Victim_race,Victim_sex,Latitude,Longitude,Lat_Lon
0,48.0,BRONX,2019-06-25,2019-07-28 14:15:00,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,M,25-44,BLACK,F,40.853355,-73.881668,"(40.85335545600003, -73.88166837799997)"
1,14.0,MANHATTAN,2019-06-25,2019-07-28 13:55:00,341,MISDEMEANOR,PETIT LARCENY,45-64,BLACK,M,UNKNOWN,UNKNOWN,D,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)"
2,18.0,MANHATTAN,2019-06-25,2019-07-28 13:20:00,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,NaN,NaN,NaN,45-64,WHITE,M,40.761677,-73.974917,"(40.761676504000036, -73.97491700299997)"
3,52.0,BRONX,2019-06-25,2019-07-28 13:20:00,104,FELONY,RAPE,45-64,BLACK HISPANIC,M,25-44,WHITE HISPANIC,F,40.869059,-73.879630,"(40.86905853200005, -73.87963014799993)"
4,19.0,MANHATTAN,2019-06-25,2019-07-28 13:20:00,578,VIOLATION,HARRASSMENT 2,25-44,BLACK,M,45-64,BLACK,M,40.783426,-73.949163,"(40.78342557000008, -73.94916258199999)"


In [24]:
# Check the number of null or NaN
complaints.isnull().sum(axis = 0)

Address_precinct        5
Hood                  138
Complaint_date          0
Complaint_time          0
Key_code                0
Offense_level           0
Offense_desc            9
Suspect_age         52382
Suspect_race        52382
Suspect_sex         52382
Victim_age              0
Victim_race             0
Victim_sex              0
Latitude               22
Longitude              22
Lat_Lon                22
dtype: int64

In [ ]:
"""
I'm not removing the NaN or Null values right now. I will see what I do with them later.
"""

In [25]:
# Import the complaints table to a csv file called complaints.csv
complaints.to_csv("complaints.csv")